In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/credit-debit dataset.csv
/kaggle/input/dataset/TransactionDataset1.csv


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# Load your dataset (replace 'your_dataset.csv' with the actual file path)
df = pd.read_csv('/kaggle/input/dataset/TransactionDataset1.csv')

# Select features (X) and target variable (y)
X = df[[ 'transaction_time',
         ]]

y = df['fraud_indicator']



In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

# Assuming X and y are your feature matrix and target variable, respectively
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define numerical and categorical columns
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Create transformers for numerical and categorical columns
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Use ColumnTransformer to apply transformers to the correct columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Fit and transform X_train
X_train_transformed = preprocessor.fit_transform(X_train)

# Get the number of features after transformation
num_features = X_train_transformed.shape[1]

# Convert the sparse matrix to a dense array
X_train_transformed_dense = X_train_transformed.toarray()

# Reshape X_train to include the time dimension
X_train_reshaped = X_train_transformed_dense.reshape((X_train_transformed_dense.shape[0], 1, num_features))

# Build the Bidirectional LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(64, activation='relu'), input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with verbose set to 1
model.fit(X_train_reshaped, y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1)

# Transform X_test
X_test_transformed = preprocessor.transform(X_test)

# Convert the sparse matrix to a dense array
X_test_transformed_dense = X_test_transformed.toarray()

# Reshape X_test to include the time dimension
X_test_reshaped = X_test_transformed_dense.reshape((X_test_transformed_dense.shape[0], 1, num_features))

# Evaluate the model on the test set
accuracy = model.evaluate(X_test_reshaped, y_test, verbose=1)
print(f'Test Accuracy: {accuracy[1]*100:.2f}%')

# Make predictions on the test set
y_pred = (model.predict(X_test_reshaped) > 0.5).astype(int)


Epoch 1/5
400/400 [==============================] - 11s 24ms/step - loss: 0.6628 - accuracy: 0.6323 - val_loss: 0.6537 - val_accuracy: 0.6394
Epoch 2/5
400/400 [==============================] - 9s 21ms/step - loss: 0.6101 - accuracy: 0.6341 - val_loss: 0.6556 - val_accuracy: 0.6384
Epoch 3/5
400/400 [==============================] - 9s 22ms/step - loss: 0.3417 - accuracy: 0.9400 - val_loss: 0.7254 - val_accuracy: 0.6250
Epoch 4/5
400/400 [==============================] - 9s 22ms/step - loss: 0.1033 - accuracy: 0.9600 - val_loss: 0.7934 - val_accuracy: 0.6259
Epoch 5/5
125/125 [==============================] - 1s 4ms/step - loss: 0.8085 - accuracy: 0.6298
Test Accuracy: 62.98%
125/125 [==============================] - 1s 4ms/step


In [ ]:

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')
